In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

In [9]:
# the data, split between train and test sets
# x_train: feature matrix for 6000 images. Shape: (6000, 28, 28)
# y_train: label array for 6000 images. Shape: (6000)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train[133]

9

In [6]:
# Tensorflow is channels_last
# Keras refers to this as backend. We need not worry about it right now.
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # this will get executed
    # mnist.load_data() supplies the MNIST digits with structure (nb_samples, 28, 28)
    # i.e. with 2 dimensions per example representing a greyscale image 28x28.
    # The Convolution2D layers in Keras however, are designed to work with 3 dimensions per example.
    # The greyscale image for MNIST digits input would either need a different CNN layer design 
    # (or a param to the layer constructor to accept a different shape), or the design could simply
    # use a standard CNN and you must explicitly express the examples as 1-channel images. 
    # The Keras team chose the latter approach, which needs the re-shape.
    # Learn more: 
    # https://datascience.stackexchange.com/questions/11704/reshaping-of-data-for-deep-learning-using-keras
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# NEW shape
# x_train: (6000, 28, 28, 1)

In [26]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizing data: As expected, the pixel values range from 0 to 255
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [27]:
# convert class vectors to binary class matrices
# np.utils.to_categorical is used to convert array of labeled data
# (from 0 to nb_classes-1) to one-hot vector.
# Arguments
# y: class vector to be converted into a matrix
# nb_classes: total number of classes
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# NEW shape:
# y_train: (6000, 10)

(60000, 10)

In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.2570 - accuracy: 0.9203 - val_loss: 0.0638 - val_accuracy: 0.9802
Epoch 2/12
60000/60000 [==============================] - 120s 2ms/step - loss: 0.0899 - accuracy: 0.9730 - val_loss: 0.0502 - val_accuracy: 0.9820
Epoch 3/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0672 - accuracy: 0.9800 - val_loss: 0.0337 - val_accuracy: 0.9888
Epoch 4/12
60000/60000 [==============================] - 110s 2ms/step - loss: 0.0548 - accuracy: 0.9834 - val_loss: 0.0318 - val_accuracy: 0.9899
Epoch 5/12
60000/60000 [==============================] - 96s 2ms/step - loss: 0.0475 - accuracy: 0.9856 - val_loss: 0.0316 - val_accuracy: 0.9893
Epoch 6/12
60000/60000 [==============================] - 120s 2ms/step - loss: 0.0420 - accuracy: 0.9872 - val_loss: 0.0284 - val_accuracy: 0.9909
Epoch 7/12
60000/60000 [==============================] - 108s

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02408905099705862
Test accuracy: 0.9921000003814697
